### Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
  - оценка 2 и меньше - низкий рейтинг
  - оценка 4 и меньше - средний рейтинг
  - оценка 4.5 и 5 - высокий рейтинг
Результат классификации запишите в столбец class

In [18]:
import pandas as pd

In [152]:
data = pd.read_csv('movies.csv')
data_ratings = pd.read_csv('ratings.csv')
#data_ratings.head()

In [153]:
movie_mean_rat = data_ratings.groupby('movieId')
movie_mean_rat = movie_mean_rat.rating.mean().reset_index()

title_mean_rat = data.merge(movie_mean_rat, how='right', left_on='movieId', right_on='movieId')

title_mean_rat.rename(columns={'rating': 'mean_rat'}, inplace=True)
#title_mean_rat.head()

In [155]:
def level_rat(class_):
    if class_['mean_rat'] >= 4.5:
        output = 'высокий рейтинг'
    elif class_['mean_rat'] <= 2.5:
        output = 'низкий рейтинг'
    else:
        output = 'средний рейтинг'
    return output

In [156]:
title_mean_rat['class'] = title_mean_rat.apply(level_rat, axis = 1)

In [157]:
# проверка корректности определения уровн рейтинга
title_mean_rat[title_mean_rat['class'] == 'средний рейтинг'].head()

,movieId,title,genres,mean_rat,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,средний рейтинг
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869,средний рейтинг
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017,средний рейтинг
4,5,Father of the Bride Part II (1995),Comedy,3.267857,средний рейтинг
5,6,Heat (1995),Action|Crime|Thriller,3.884615,средний рейтинг


### Задание 2**
Используем файл keywords.csv.
Необходимо написать гео-классификатор: каждой строке выставить принадлежность региону. 

Если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. 
Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Результат классификации запишите в отдельный столбец region.

In [85]:
# Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

## Хотел предусмотреть разные написания и использовать pymystem и потом посредством list и set искать наличие пересечений списков, 
# но преобразование строк уходит в бесконечный цикл

geo_data = {
    'Центр': ['москв', 'тул', 'ярославл'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [20]:
data_kw = pd.read_csv('keywords.csv')
data_kw.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [89]:
def check_city(keyword):
    
    for key, value in geo_data.items():
        for i in value:
            if i in keyword:
                return key
            
    return 'undefined'

In [91]:
data_kw['region'] = data_kw['keyword'].apply(check_city)
data_kw[data_kw.region == 'Центр'].head()

,keyword,shows,region
127,авито москва,979292,Центр
143,эхо москвы,889657,Центр
197,погода в москве,745745,Центр
414,погода в москве на 14 дней,400914,Центр
512,карта метро москвы,331857,Центр


### Задание 3 (бонусное)
Проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
Файл с рейтингами фильмов ratings.csv из базы https://grouplens.org/datasets/movielens

Алгоритм:

1. В переменную years запишите список из всех годов с 1950 по 2010.
2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. 

Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

* для каждой строки пройдите по всем годам списка years
- если номер года присутствует в названии фильма - return год выпуска
- если нет номеров года списка years в названии фильма - 1900 год
* запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

3. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [26]:
import pandas as pd
import numpy as np

In [12]:
years = list(range(1950, 2011))

In [30]:
ratings = pd.read_csv('ratings.csv')
data_films = pd.read_csv('movies.csv')
rat_mov = ratings.merge(data_films, how = 'inner', left_on='movieId', right_on='movieId')
rat_mov.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,7,31,3.0,851868750,Dangerous Minds (1995),Drama
2,31,31,4.0,1273541953,Dangerous Minds (1995),Drama
3,32,31,4.0,834828440,Dangerous Minds (1995),Drama
4,36,31,3.0,847057202,Dangerous Minds (1995),Drama


In [95]:
def production_year(row):

    for i in years:
        if str(i) in row:
            return i
    return 1900

In [96]:
rat_mov['year'] = rat_mov['title'].apply(production_year)

In [108]:
group_year = rat_mov.groupby('year')
group_year.rating.mean().reset_index().sort_values('rating', ascending=False).head(50)
#group_year.rating.mean()

,year,rating
8,1957,4.014241
23,1972,4.011136
3,1952,4.000000
5,1954,3.992754
2,1951,3.983539
25,1974,3.978625
13,1962,3.952316
1,1950,3.915254
28,1977,3.905506
10,1959,3.841033
